In [1]:
import os
import pandas as pd
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import time

In [2]:
with open("res.json") as file:
    data = pd.read_json(file)
data.shape
with open("res_france.json") as file:
    data_france = pd.read_json(file)
data.shape
with open("res_italy.json") as file:
    data_italy = pd.read_json(file)
data.shape
with open("res_usa.json") as file:
    data_US = pd.read_json(file)
data.shape
with open("res_germany.json") as file:
    data_germany = pd.read_json(file)

In [3]:
data=data[data['brand']!="ABB"]
data=data[data['brand']!="Callebaut"]
data=data[data['brand']!="ABB"]
data=data[data['brand']!="STMicroelectronics"]
data=data[(data['brand']!="IWC")|((data['brand']=="IWC") & \
                    ((data["categories"]!="AutomotivePaint, Body & TrimBodyDoors")& \
                                (data["categories"]!='AutomotiveTires & WheelsAccessoriesCenter Caps')& \
                                    (data["categories"]!='AutomotiveTires & WheelsHubcaps & CoversHubcaps')))]
data=data[(data['brand']!="Swatch")|((data['brand']=="Swatch") & \
                    ((data["categories"]!="Arts, Crafts & SewingFabric")& \
                                    (data["categories"]!='Arts, Crafts & SewingNeedleworkCross-StitchCounted Kits')))]
data=data[(data['brand']!="Sigg")|((data['brand']=="Sigg") & \
                                    (data["categories"]!='Office ProductsOffice & School SuppliesForms, Recordkeeping & Money HandlingMoney & Rent Receipts'))]
data=data[(data['brand']!="PB Swiss")|((data['brand']=="PB Swiss") & \
                    ((data["categories"]!="Clothing, Shoes & JewelryNovelty, Costumes & MoreNoveltyJewelryBracelets")& \
                                (data["categories"]!='Office ProductsOffice & School SuppliesCalendars, Planners & Personal OrganizersWall Calendars')& \
                                    (data["categories"]!='Sports & OutdoorsAccessoriesWristbands')))]
data=data[(data['brand']!="Omega")|((data['brand']=="Omega") & \
                    ((data["categories"]=="Clothing, Shoes & JewelryGifts")| \
                    (data["categories"]=='Clothing, Shoes & JewelryJewelry: International Shipping Available')| \
                    (data["categories"]=='Clothing, Shoes & JewelryLuxury Watches')| \
                     (data["categories"]=='Clothing, Shoes & JewelryNew Arrivals')| \
                     (data["categories"]=='Clothing, Shoes & JewelryOmega Watches')| \
                     (data["categories"]=='Sports & OutdoorsAccessoriesSport Watches')))]

In [4]:
data.head(5)

,asin,brand,categories,description,helpful,imUrl,overall,price,related,reviewText,reviewTime,reviewerID,reviewerName,salesRank,summary,title,unixReviewTime
0,B0009YJ238,Nestle,"[[Pet Supplies, Dogs, Treats, Cookies, Biscuit...","Busy Bone, 7 OZ, Small/Medium Dog Treat, Twist...","[0, 0]",http://ecx.images-amazon.com/images/I/31ToYepJ...,5,3.49,NaN,"I adore my six-month-old dog, Midge. She's ev...","01 22, 2007",A3MTXMDJ7JQSFO,A. Ruminski,{'Pet Supplies': 487454},Busy bone = happy owner,BusyBone MED Dog Treat,1169424000
1,B00126U31C,Swatch,"[[Clothing, Shoes & Jewelry, New Arrivals], [C...",NaN,"[0, 1]",http://ecx.images-amazon.com/images/I/51fwuvsL...,3,72.14,"{'also_viewed': ['B002DQGUI0', 'B000KNE09E', '...","The watch itself is solid, however, the leathe...","12 14, 2012",AYKBRA7MKIPAV,G.Jackson,{'Watches': 95452},Swatch watch,Swatch Men's STYNS421 Wild Roar Brown Dial Watch,1355443200
2,B002HHGF5I,Tissot,"[[Sports & Outdoors, Accessories, Sport Watche...",NaN,"[4, 5]",http://ecx.images-amazon.com/images/I/516lmovC...,5,846.00,"{'also_viewed': ['B0029SJ3ZE', 'B001CBC2I4', '...",incredible watch. super light yet rugged. i ...,"03 7, 2011",A3MBOY1IA8PS85,"M. Klein ""Gras""",{'Watches': 27798},great watch,Tissot Men's T0134204405700 T-Touch Expert Pil...,1299456000
3,B002HHGF5I,Tissot,"[[Sports & Outdoors, Accessories, Sport Watche...",NaN,"[8, 11]",http://ecx.images-amazon.com/images/I/516lmovC...,5,846.00,"{'also_viewed': ['B0029SJ3ZE', 'B001CBC2I4', '...",I have been a watch junkie for some years but ...,"02 27, 2010",A1SE5JXA03WBDT,"Mem Ber ""me""",{'Watches': 27798},the best watch in my collection,Tissot Men's T0134204405700 T-Touch Expert Pil...,1267228800
4,B002HHGF5I,Tissot,"[[Sports & Outdoors, Accessories, Sport Watche...",NaN,"[0, 2]",http://ecx.images-amazon.com/images/I/516lmovC...,5,846.00,"{'also_viewed': ['B0029SJ3ZE', 'B001CBC2I4', '...",just what I wanted when I ordered this watch. ...,"01 7, 2014",A4ACF44TDCIZK,MELANIE DAVID,{'Watches': 27798},Tissot Men's T0134204405700 T-Touch Expert Pil...,Tissot Men's T0134204405700 T-Touch Expert Pil...,1389052800


If we inspect some entries, it can be seen that the category column for some of the rows contains more than one entries. See example below

In [5]:
data['categories'][1]

[['Clothing, Shoes & Jewelry', 'New Arrivals'],
 ['Clothing, Shoes & Jewelry', 'Jewelry: International Shipping Available'],
 ['Clothing, Shoes & Jewelry', 'Fashion Watches'],
 ['Clothing, Shoes & Jewelry', 'Available for International Shipping'],
 ['Clothing, Shoes & Jewelry', 'Swatch Watches'],
 ['Clothing, Shoes & Jewelry', 'Men', 'Watches', 'Wrist Watches']]

We figure out the number of such entries (that we call invalid) in our dataset

In [ ]:
data['flag'] = data['categories'].apply(lambda x: False if len(x)>1 else True)
print(data.flag.value_counts())
print('Number of invalid entries: ', data.flag.value_counts()[0] ,'\n',
      'Percentage of total entries that are invalid: ',
      100*data.flag.value_counts()[0] / data.flag.value_counts().sum())

True     199104
False      8288
Name: flag, dtype: int64
Number of invalid entries:  8288 
 Percentage of total entries that are invalid:  3.99629686777


We intend to perform the category-wise analysis of our reviews and since these entries (that are slightly complicated to handle due to multiple values for category attribute) happen to be only 4% of the entire dataset, we drop them.

In [ ]:
import numpy as np
def delete_more_categories(data):
    data['flag'] = data['categories'].apply(lambda x: False if len(x)>1 else True)
    return data[data['flag']]
data = delete_more_categories(data)
data_france = delete_more_categories(data_france)
data_italy = delete_more_categories(data_italy)
data_US = delete_more_categories(data_US)
data_germany = delete_more_categories(data_germany)

In [ ]:
print(data['categories'][0])

Our objective now is to determine a level in the hierarchy of categories where we should stop, so that the different number of categories we get are manageable for analysis and for interpretable results. 

For example the categories attribute for the 154-th instance printed above is a 5 member list -  
['Home & Kitchen', 'Kitchen & Dining', 'Small Appliances', 'Juicers', 'Masticating Juicers']

If we decide to stop at a hierarchy level of 1, we would clip this list to simply ['Home & Kitchen'] or if we stop at the second level we'll end up with ['Home & Kitchen', 'Kitchen & Dining'] for this instance. A good choice of what level to stop at will be guided by the number of unique category attributes we have to deal with for the given level. To begin with, we stop at a level of 2 to realize that we get 103 unique categories which are too much for proper interpretability, hence we go up the level and check for 1.

In [ ]:
df_test = data.copy()
df_test['categories_clipped'] = df_test['categories'].apply(lambda x: (x[0][0] + ', ' + x[0][1]) if len(x[0])>2 
                                                            else x[0][0] )
print('Number of unique categories after clipping:', len(df_test.categories_clipped.unique()),
      '\n',df_test.categories_clipped.value_counts())

In [ ]:
def cut_layers(data): 
    data['categories'] = data['categories'].apply(lambda x: x[0][0])
cut_layers(data)
cut_layers(data_france)
cut_layers(data_italy)
cut_layers(data_US)
cut_layers(data_germany)

In [ ]:
print(data['categories'][0])

As shown in the cell below, if we simply clip to level 1, we get 25 unique categories where 12 of them have less than 1000 reviews. In our further steps we stick to the only 13 categories that have atleast 1000 reviews, so that our claims are not based on small number of reviews.

In [ ]:
print('Number of unique categories after clipping:', len(data.categories.unique()),
      '\n','Number of unique categories with more than 1000 reviews:',
      (data.categories.value_counts() > 1000).value_counts()[1],
      '\n',data.categories.value_counts())

We intend to analyze the variations of ratings for these different categories over periods of time since we also access to the time of the review. Let's begin with printing the average ratings obtained for these categories.

In [ ]:
import datetime as dt
print(dt.datetime.utcfromtimestamp(df.unixReviewTime[0]).strftime("%Y/%m/%d %H:%M"),
dt.datetime.utcfromtimestamp(df.unixReviewTime[213872]).strftime("%Y/%m/%d %H:%M"),
dt.datetime.utcfromtimestamp(df.unixReviewTime[160]).strftime("%Y/%m/%d %H:%M"))

As it turns out, all the unixtime values transform to '00:00' hours with the date matching to the column 'reviewTime', unfortunately the 'unixReviewTime' does not provide additional information for the time of the day when the review was made.

We now play with one of the category ('Home & Kitchen') and analyze the variations of rating over time for this category.

In [ ]:
categories=list(data["categories"].drop_duplicates())
category_group = data.groupby("categories")
brand_group = data.groupby("brand")
france_grouped = data_france.groupby("categories")
italy_grouped = data_italy.groupby("categories")
usa_grouped = data_US.groupby("categories")
germany_grouped = data_germany.groupby("categories")
categories_france=list(data_france["categories"].drop_duplicates())
categories_italy=list(data_italy["categories"].drop_duplicates())
categories_usa=list(data_US["categories"].drop_duplicates())
categories_germany=list(data_germany["categories"].drop_duplicates())

In [ ]:
category_sorted=sorted(category_group, key=lambda x: len(x[1]), reverse=True)
brand_sorted=sorted(brand_group, key=lambda x: len(x[1]), reverse=True)
cat_means={}
bran_means={}
significant_categories = [category_sorted[i][0] for i in range(13)]
significant_brands = [brand_sorted[i][0] for i in range(10)]

In [ ]:
eg={}
for category_name in significant_categories:
    eg[category_name] = data[data['categories'] == category_name].copy()
    eg[category_name].sort_values( 'unixReviewTime', axis=0,inplace = True)
    eg[category_name]['reviewMonth'] = ((eg[category_name].unixReviewTime).map(lambda t: datetime.fromtimestamp(t).month))
    eg[category_name]['reviewYear']  = ((eg[category_name].unixReviewTime).map(lambda t: datetime.fromtimestamp(t).year))

In [ ]:
eg_brand={}
for brand_name in significant_brands:
    eg_brand[brand_name] = data[data['brand'] == brand_name].copy()
    eg_brand[brand_name].sort_values( 'unixReviewTime', axis=0,inplace = True)
    eg_brand[brand_name]['reviewMonth'] = ((eg_brand[brand_name].unixReviewTime).map(lambda t: datetime.fromtimestamp(t).month))
    eg_brand[brand_name]['reviewYear']  = ((eg_brand[brand_name].unixReviewTime).map(lambda t: datetime.fromtimestamp(t).year))

In [ ]:
#Mean rating of each brand
#make weighted statistic of data - take overall rate according to helpfulness (how helpful is rating and comment)
def calculate_review(data_frame):
    s=[]
    overall=[]
    overall_count=0
    for index, row in data_frame.iterrows():
        if row["helpful"][1]!=0: #and row["helpful"][1]>=3:
            if row["helpful"][0]!=0:
                overall.append(row["overall"]*(row["helpful"][0]/row["helpful"][1]))
                overall_count+=row["helpful"][0]/row["helpful"][1]
            else: 
                overall.append(row["overall"]*(2/(2+row["helpful"][1])))
                overall_count+=(2/(2+row["helpful"][1]))
        else: 
            overall.append(row["overall"])
            overall_count+=1
    if overall_count!=0: return sum(overall)/overall_count, overall_count
    else: return 0, overall_count

In [ ]:
#what are the most and the least popular brands within each category
from operator import itemgetter
import operator
import numpy as np
def popular_brands(cat, data):
    categories = data["categories"].drop_duplicates()
    grouped_cat = data.groupby("categories")
    category = pd.DataFrame(grouped_cat.get_group(cat))
    brands = category["brand"].drop_duplicates()
    grouped_brands = category.groupby("brand")
    reviews={}
    for b in brands:
        brand = pd.DataFrame(grouped_brands.get_group(b))
        if calculate_review(product)[0]!=0: reviews[b]=calculate_review(brand)
    return sorted(reviews.items(), key=operator.itemgetter(1), reverse=True)
#popular_brands("Electronics", data)

In [ ]:
list(category_group.groups.keys())

In [ ]:
for x in significant_categories: cat_means[x]=calculate_review(pd.DataFrame(category_group.get_group(x)))[0]
for x in significant_brands: bran_means[x]=calculate_review(pd.DataFrame(brand_group.get_group(x)))[0]

In [ ]:
#what are the best rated products of a brand
from operator import itemgetter
import operator
import numpy as np
best={}
worst={}
for b in significant_brands:
    brand = pd.DataFrame(brand_group.get_group(b))
    products = brand["asin"].drop_duplicates()
    grouped_products = brand.groupby("asin")
    reviews={}
    for p in products:
        product = pd.DataFrame(grouped_products.get_group(p))
        if calculate_review(product)[0]!=0: reviews[p]=calculate_review(product)
    brand_products = sorted(reviews.items(), key=operator.itemgetter(1), reverse=True)
    i=1
    while brand_products[-i][1][1]==1: i+=1
    best[b]=brand_products[0][0]
    worst[b]=brand_products[-i][0]
    print(b + " " + brand_products[0][0] + " " + brand_products[-i][0])

In [ ]:
sorted_reviews={}
all_data={}
all_data_brand={}
for c in significant_categories: sorted_reviews[c] = popular_brands(c, data)
for c in significant_categories: all_data[c] = [list(pd.DataFrame(category_group.get_group(c))["overall"])]
for b in significant_brands: all_data_brand[b] = [list(pd.DataFrame(brand_group.get_group(b))["overall"])]

In [ ]:
france_means={}
italy_means={}
usa_means={}
germany_means={}
for category in significant_categories:
    if category in categories_france: 
        france_means[category]=calculate_review(pd.DataFrame(france_grouped.get_group(category)))[0]
for category in significant_categories:
    if category in categories_italy: 
        italy_means[category]=calculate_review(pd.DataFrame(italy_grouped.get_group(category)))[0]
for category in significant_categories:
    if category in categories_usa: 
        usa_means[category]=calculate_review(pd.DataFrame(usa_grouped.get_group(category)))[0]
for category in significant_categories:
    if category in categories_germany: 
        germany_means[category]=calculate_review(pd.DataFrame(germany_grouped.get_group(category)))[0]

#slucaj 18/19 obavezno napomenuti B00ADBY92Q
#22/29 1 B003YFHWP4

In [ ]:
from IPython.html.widgets import interact, fixed
import numpy as np
import operator
from operator import itemgetter
#%matplotlib qt
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display

def plot_rir(DEAL_WITH_CATEGORIES = False,  
                                             category="Electronics",
                                            brands_within_category=False,
                                             change_of_ratings_over_time=False,
                                             compare_with_other_countries=False, 
                                            DEAL_WITH_BRANDS = False,  
                                             brand="Victorinox", show_best_and_worst_product=False,
                                             avg_price_over_time=False
                                            ): #s=None, length=30000, number=8, , 
    global all_data, france_means, italy_means, usa_means, germany_means, all_data_brand
    if DEAL_WITH_CATEGORIES == True:
        
        if brands_within_category==False:
            categories_names=significant_categories
            categories_means = []
            for x in significant_categories: categories_means.append(float(cat_means[x]))
            pairs = [(x,y) for x,y in zip(categories_names,categories_means)]
            pairs = sorted(pairs, key=itemgetter(1))
            brands_names,brands_means = zip(*pairs)
            X=range(0,len(brands_names),1)
            brands_means=brands_means[0::1]
            plt.figure(figsize = (8, 4))
            plt.bar(X, brands_means, color='g', align='center', width=0.5)
            plt.xticks(X, brands_names[0::1], rotation='vertical')    
            plt.title("Ratings of Most Categories with most Reviews")
            plt.xlabel('Categories with most Reviews')    
            plt.ylabel("Ratings")  
        
        if brands_within_category==False:
            fig, axes = plt.subplots(nrows=1, figsize=(10, 6))
            #axes.boxplot(all_data[category], showfliers=True)
            #axes.set_title('box plot')
            axes.violinplot(all_data[category], showmeans=True, showmedians=True)
            axes.set_title('violin plot')
            # adding horizontal grid lines
            axes.yaxis.grid(True)
            #ax.set_xticks([y+1 for y in range(len(all_data))])
            axes.set_xlabel('xlabel')
            axes.set_ylabel('ylabel')
            # add x-tick labels
            plt.setp(axes, xticks=[y+1 for y in range(1)], xticklabels=["category: "+ category])
            plt.show()
            
        if brands_within_category == True:
            brands_names=[]
            brands_means=[]
            for x in range(len(sorted_reviews[category])): brands_names.append(sorted_reviews[category][x][0])
            for x in range(len(sorted_reviews[category])): brands_means.append(float(sorted_reviews[category][x][1][0]))
            pairs = [(x,y) for x,y in zip(brands_names,brands_means)]
            pairs = sorted(pairs, key=itemgetter(1))
            brands_names,brands_means = zip(*pairs)
            X=range(0,len(brands_names),1)
            brands_means=brands_means[0::1]
            plt.figure(figsize = (8, 4))
            plt.bar(X, brands_means, color='m', align='center', width=0.7)
            plt.xticks(X, brands_names[0::1], rotation='vertical')    
            plt.title("Ratings of Brands within Category")
            plt.xlabel('Brands within Category')    
            plt.ylabel("Ratings")    
        if change_of_ratings_over_time==True:
            hor_axis = []
            for index in eg[category].groupby(['reviewYear']).mean()['unixReviewTime'].index:
                Y = eg[category].groupby(['reviewYear']).mean()['unixReviewTime'].map(lambda t: datetime.fromtimestamp(t))[index].year
                dt = datetime(year=Y, month=1, day=1)
                new_year_day = time.mktime(dt.timetuple())
                hor_axis.append(datetime.fromtimestamp(new_year_day))
            plt.rcParams["figure.figsize"] = [8,3]
            years = mdates.YearLocator()   # every year
            months = mdates.MonthLocator()  # every month
            yearsFmt = mdates.DateFormatter('%Y')
            fig, ax = plt.subplots()
            ax.plot( eg[category].groupby(['reviewYear','reviewMonth']).mean()['unixReviewTime'].map
                     (lambda t: datetime.fromtimestamp(t)),
                     eg[category].groupby(['reviewYear','reviewMonth']).mean()['overall'],'y',label='Average Monthly Rating')
            ax.plot(hor_axis, eg[category].groupby(['reviewYear']).mean()['overall'],'ro-',label='Average Annual Rating')
            # format the ticks
            ax.xaxis.set_major_locator(years)
            ax.xaxis.set_major_formatter(yearsFmt)
            ax.xaxis.set_minor_locator(months)
            ax.grid(True)
            plt.title((category))
            plt.legend(loc='lower left')
            plt.show()
        
        if compare_with_other_countries==True: 
            countries_names=["Switzerland"]
            countries_means=[cat_means[category]]
            if category in france_means:
                countries_names.append("France")
                countries_means.append(france_means[category])
            if category in italy_means:
                countries_names.append("Italy")
                countries_means.append(italy_means[category])
            if category in usa_means:
                countries_names.append("USA")
                countries_means.append(usa_means[category])
            if category in germany_means:
                countries_names.append("Germany")
                countries_means.append(germany_means[category])
            pairs = [(x,y) for x,y in zip(countries_names,countries_means)]
            pairs = sorted(pairs, key=itemgetter(1))
            brands_names,brands_means = zip(*pairs)
            X=range(0,len(brands_names),1)
            brands_means=brands_means[0::1]
            plt.figure(figsize = (8, 4))
            plt.bar(X, brands_means, color='b', align='center', width=0.7)
            plt.xticks(X, brands_names[0::1], rotation='vertical')    
            plt.title("Average Ratings of other Countries within Category: " + category)
            plt.xlabel('Countries')    
            plt.ylabel("Ratings") 
            
            
    if DEAL_WITH_BRANDS==True:
        brands_names=significant_brands
        brands_means=[]
        for x in significant_brands: brands_means.append(float(bran_means[x]))
        pairs = [(x,y) for x,y in zip(brands_names,brands_means)]
        pairs = sorted(pairs, key=itemgetter(1))
        brands_names,brands_means = zip(*pairs)
        X=range(0,len(brands_names),1)
        brands_means=brands_means[0::1]
        plt.figure(figsize = (8, 4))
        plt.bar(X, brands_means, color='r', align='center', width=0.7)
        plt.xticks(X, brands_names[0::1], rotation='vertical')    
        plt.title("Ratings of Brands with most Reviews")
        plt.xlabel('Brands with most Reviews')    
        plt.ylabel("Ratings") 
        
        url_b='./'+brand+'/best.jpg'
        url_w='./'+brand+'/worst.jpg'
            
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 6))
        #axes[0].boxplot(all_data_brand[brand], showfliers=True)
        #axes[0].set_title('box plot')
        axes.violinplot(all_data_brand[brand], showmeans=True, showmedians=True)
        axes.set_title('violin plot')
        # adding horizontal grid lines
        axes.yaxis.grid(True)
        #ax.set_xticks([y+1 for y in range(len(all_data_brand))])
        axes.set_xlabel('xlabel')
        axes.set_ylabel('ylabel')
        # add x-tick labels
        plt.setp(axes, xticks=[y+1 for y in range(1)], xticklabels=["brand: "+ brand])
        plt.show()
            
        if show_best_and_worst_product == True:
            i=Image(url_b, width=400,height=300)
            display(i)
            pd.options.display.max_colwidth = 100
            title = str(data[data["asin"]==best[brand]]["title"].drop_duplicates())
            title=title[:title.find(",")]
            print(''.join([i for i in title if not i.isdigit()]))
            i=Image(url_w, width=400,height=300)
            display(i)
            title=str(data[data["asin"]==worst[brand]]["title"].drop_duplicates())
            title = title[:title.find(",")]
            print(''.join([i for i in title if not i.isdigit()]))
        
        if avg_price_over_time==True:
            hor_axis = []
            for index in eg_brand[brand].groupby(['reviewYear']).mean()['unixReviewTime'].index:
                Y = eg_brand[brand].groupby(['reviewYear']).mean()['unixReviewTime'].map(lambda t: datetime.fromtimestamp(t))[index].year
                dt = datetime(year=Y, month=1, day=1)
                new_year_day = time.mktime(dt.timetuple())
                hor_axis.append(datetime.fromtimestamp(new_year_day))
            plt.rcParams["figure.figsize"] = [8,3]
            years = mdates.YearLocator()   # every year
            months = mdates.MonthLocator()  # every month
            yearsFmt = mdates.DateFormatter('%Y')
            fig, ax = plt.subplots()
            ax.plot( eg_brand[brand].groupby(['reviewYear','reviewMonth']).mean()['unixReviewTime'].map
                     (lambda t: datetime.fromtimestamp(t)),
                     eg_brand[brand].groupby(['reviewYear','reviewMonth']).mean()['price'],'y',label='Average Monthly Rating')
            ax.plot(hor_axis, eg_brand[brand].groupby(['reviewYear']).mean()['price'],'ro-',label='Average Annual Rating')
            # format the ticks
            ax.xaxis.set_major_locator(years)
            ax.xaxis.set_major_formatter(yearsFmt)
            ax.xaxis.set_minor_locator(months)
            ax.grid(True)
            plt.title((brand))
            plt.legend(loc='upper right')
            plt.show()
        
        #input("Press enter to exit ;)")
interact(plot_rir, category=significant_categories, brand=significant_brands) #plot_rir, s=fixed(rir), length=(500, 25000, 1000), number=(1,8),